In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def reset_graph(seed):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
reset_graph(42)

Manual RNN

In [4]:
n_inputs = 3 # Vector/Feature size
n_neurons = 5 # Nodes of the only hidden layer

In [5]:
x0 = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs))
x1 = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs))

In [6]:
wx = tf.Variable(initial_value=tf.random_normal(shape=(n_inputs, n_neurons)), dtype=tf.float32)
wy = tf.Variable(initial_value=tf.random_normal(shape=(n_neurons, n_neurons)), dtype=tf.float32)

In [7]:
b = tf.Variable(tf.zeros(shape=(1, n_neurons)), dtype=tf.float32)

In [8]:
y0 = tf.tanh(tf.matmul(x0, wx) + b)

In [9]:
y1 = tf.tanh(tf.matmul(y0, wy) + tf.matmul(x1, wx) + b)

In [10]:
init = tf.global_variables_initializer()

In [11]:
x0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # At t=0, there are 4 instances fed
x1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # At t=1, the next 4 instances are fed

In [12]:
with tf.Session() as sess:
    sess.run(init)
    y0_val, y1_val = sess.run([y0, y1], feed_dict={x0: x0_batch, x1: x1_batch})

In [13]:
y0_val

array([[-0.06640061,  0.96257669,  0.68105787,  0.70918542, -0.89821595],
       [ 0.99777555, -0.71978879, -0.99657613,  0.96739244, -0.99989706],
       [ 0.99999785, -0.99898809, -0.99999887,  0.99677628, -0.99999988],
       [ 1.        , -1.        , -1.        , -0.99818921,  0.99950868]], dtype=float32)

In [14]:
y1_val

array([[ 1.        , -1.        , -1.        ,  0.40200272, -0.99999994],
       [-0.12210429,  0.62805295,  0.96718436, -0.99371219, -0.25839329],
       [ 0.99999815, -0.9999994 , -0.99999744, -0.85943311, -0.99998796],
       [ 0.99928296, -0.99999809, -0.99990588,  0.98579615, -0.92205751]], dtype=float32)

Now with TF inbuilt functions static_rnn()

In [ ]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [ ]:
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [x0, x1], dtype=tf.float32)

In [ ]:
Y0, Y1 = output_seqs

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    print(Y0.eval(feed_dict={x0: x0_batch, x1: x1_batch}))
    print(Y1.eval(feed_dict={x0: x0_batch, x1: x1_batch}))

Now further succinct 

In [ ]:
n_steps = 2

In [ ]:
X = tf.placeholder(dtype=tf.float32, shape=(None, n_steps, n_inputs))

In [ ]:
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))

In [ ]:
X_seqs

In [ ]:
basic_cell1 = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, reuse=True)

In [ ]:
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell1, X_seqs, dtype=tf.float32)

In [ ]:
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [ ]:
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7]],
    [[3, 4, 5], [0, 0, 0]],
    [[6, 7, 8], [6, 5, 4]],
    [[9, 0, 1], [3, 2, 1]],
])

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    init.run()
    output_vals = outputs.eval(feed_dict={X: X_batch})

In [ ]:
output_vals

With dynamic rnn

In [ ]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, reuse=True)
output, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

MNIST using dynamic rnn

In [ ]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_output = 10
learning_rate = 0.001

In [ ]:
"""
x = tf.placeholder(dtype=tf.float32, shape=(None, n_steps, n_inputs))

y = tf.placeholder(dtype=tf.int32, shape=(None))

basic_cell2 = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

outputs, states = tf.nn.dynamic_rnn(basic_cell2, x, dtype=tf.float32)
"""

In [ ]:
n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])
cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu, state_is_tuple=False)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)